In [ ]:
#CODES FOR DATA IMPORTATION AND CLEANING


import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.seasonal import seasonal_decompose
from sklearn.metrics import mean_squared_error
import warnings
warnings.filterwarnings('ignore')

#loading data
dfu = pd.read_csv(r"C:\Users\HP\Documents\usa_county_wise.csv")
dfx = pd.read_csv(r"C:\Users\HP\Documents\day_wise.csv")   #***
dfy = pd.read_csv(r"C:\Users\HP\Documents\worldometer_data.csv") #***
dfz = pd.read_csv(r"C:\Users\HP\Documents\country_wise_latest.csv")#***

 # Replacing NaN with 0
dfx.fillna(0, inplace=True)
dfy.fillna(0, inplace=True)
dfu.fillna(0, inplace=True)
dfz.fillna(0,inplace=True)
   #changing date formats of the dataframes
dfu['Date'] = pd.to_datetime(dfu['Date'])
dfu['Date'] = dfu['Date'].dt.strftime('%Y/%m/%d')
dfx['Date'] = pd.to_datetime(dfx['Date'])
dfx['Date'] = dfx['Date'].dt.strftime('%Y/%m/%d')

dfy.head()


In [ ]:
#SORTING OF DATA

#top countries total cases     (WORLDOMETER)

dfy.sort_values( by= [ 'TotalCases'], ascending = False , inplace=True )# Descending order for all columns

dfy.head(5)



dfy.sort_values( by= [ 'Tot Cases/1M pop'], ascending = False , inplace=False) # Descending order for all columns
dfy.head()


dfy.sort_values( by= [ 'Deaths/1M pop'], ascending = False , inplace=True )# Descending order for all columns

dfy.head()

dfz.sort_values( by= [ 'Deaths'], ascending = False , inplace=True )# Descending order for all columns
dfz.head()

dfu.sort_values( by= [ 'Deaths'], ascending = False , inplace=True )# Descending order for all columns
dfu.head()



In [ ]:
#FEATURE ENGINEERING

##### Group data by Continent and calculate sums for relevant columns
confirmed_cases = dfx.groupby('Date')[['Confirmed', 'Deaths', 'Recovered']].sum().reset_index()

# Converting the 'Date' column to datetime format
confirmed_cases['Date'] = pd.to_datetime(dfx_grouped['Date'])
confirmed_cases['Date'] = confirmed_cases['Date'].dt.strftime('%Y/%m/%d')
# Dropping the row at index 0 (if required for some reason, as per the image)
#continent_total_Deaths= continent_total_Deaths.drop(index=0)

# Display the result
confirmed_cases



##### Group data by Continent and calculate sums for relevant columns
continent_total_Deaths = dfy.groupby('Continent')[[ 'Population', 'TotalDeaths']].sum().reset_index()

# Calculating the percentage of affected cases
continent_total_Deaths['Percentage_Deaths (%)'] = (continent_total_Deaths['TotalDeaths'] / continent_total['Population']) * 100

# Dropping the row at index 0 (if required for some reason, as per the image)
continent_total_Deaths= continent_total_Deaths.drop(index=0)

# Display the result
continent_total_Deaths




# Group data by Continent and calculate sums for relevant columns
continent_total = dfy.groupby('Continent')[[ 'Population', 'TotalCases']].sum().reset_index()

# Calculating the percentage of affected cases
continent_total['Percentage Affected(%)'] = (continent_total['TotalCases'] / continent_total['Population']) * 100

# Dropping the row at index 0 (if required for some reason, as per the image)
continent_total = continent_total.drop(index=0)

# Display the result
continent_total
continent_total.sort_values(by="Percentage Affected(%)", ascending=False).head(10)




# Group data by Province_stateand calculate sums for relevant columns
province_state_total = dfu.groupby('Province_State')[[ 'Confirmed', 'Deaths']].sum().reset_index()


# Dropping the row at index 0 (if required for some reason, as per the image)
province_state_total = province_state_total.drop(index=0)

# Display the result
province_state_total.sort_values( by= [ 'Deaths'], ascending = False , inplace=True )# Descending order for all columns
province_state_total.head()








In [ ]:
#FOR VISUALIZATION

import matplotlib.pyplot as plt
import seaborn as sns
# Grouping data by Date and aggregating the relevant columns
dfx_grouped = dfx.groupby('Date')[['Confirmed', 'Deaths', 'Recovered']].sum().reset_index()
# Converting the 'Date' column to datetime format
dfx_grouped['Date'] = pd.to_datetime(dfx_grouped['Date'])

# Plotting the data
plt.figure(figsize=(12, 6))
plt.plot(dfx_grouped['Date'], dfx_grouped['Confirmed'], label='Confirmed Cases', color='orange')
plt.plot(dfx_grouped['Date'], dfx_grouped['Deaths'], label='Deaths', color='purple')
plt.plot(dfx_grouped['Date'], dfx_grouped['Recovered'], label='Recoveries', color='green')

# Adding title and labels
plt.title("COVID-19 Trends Over Time")
plt.xlabel('Date')
plt.ylabel('Count')

# Displaying the legend, grid, and showing the plot
plt.legend()
plt.grid()
plt.show()


# Sort the data by TotalDeaths in descending order and select the top 10
top_countries = dfy.sort_values(by="TotalDeaths", ascending=False).head(10)

# Plot the bar chart
plt.figure(figsize=(10, 6))
plt.bar(top_countries['Country/Region'], top_countries['TotalDeaths'], color='blue', alpha=0.7)

# Add titles and labels
plt.title('Top 10 Countries by Total Deaths', fontsize=16)
plt.xlabel('Country', fontsize=12)
plt.ylabel('Total Deaths', fontsize=12)
plt.xticks(rotation=45)  # Rotate x-axis labels for better readability
plt.grid(axis='y', linestyle='--', alpha=0.6, color='green')  # Add grid lines
plt.tight_layout()  # Adjust layout for better appearance
plt.show()




# Sort the data by TotalDeaths in descending order and select the top 10
top_continents = dfy.sort_values(by="TotalDeaths", ascending=False).head(10)

# Plot the bar chart
plt.figure(figsize=(10, 6))
plt.bar(top_continents['Continent'], top_continents['TotalDeaths'], color='blue', alpha=0.7)

# Add titles and labels
plt.title('Top 10 continents by Total Deaths', fontsize=16)
plt.xlabel('Continent', fontsize=6)
plt.ylabel('Total Deaths', fontsize=6)
plt.xticks(rotation=45)  # Rotate x-axis labels for better readability
plt.grid(axis='y', linestyle='--', alpha=0.6, color='green')  # Add grid lines
plt.tight_layout()  # Adjust layout for better appearance
plt.show()





# Plotting a pie chart
plt.figure(figsize=(8, 8))
plt.pie(
    continent_total['Percentage Affected(%)'],
    labels=continent_total['Continent'],
    autopct='%1.1f%%',
    startangle=90,
    colors=plt.cm.tab10.colors 
)



# Plotting a pie chart
plt.figure(figsize=(8, 8))
plt.pie(
    continent_total_Deaths['Percentage_Deaths (%)'],
    labels=continent_total['Continent'],
    autopct='%1.1f%%',
    startangle=90,
    colors=plt.cm.tab10.colors  # Optional: custom colors
)


import plotly.graph_objects as go

# Selecting top 10 provinces/states based on deaths for better visualization
top_provinces = province_state_total.head(10)

# Creating the bar chart
fig = go.Figure()

# Adding 'Confirmed' data
fig.add_trace(go.Bar(
    x=top_provinces['Province_State'], 
    y=top_provinces['Confirmed'], 
    name='Confirmed', 
    marker_color='yellow',
    hoverinfo='x+y',  # Show Province_State and Confirmed values on hover
    text=top_provinces['Confirmed'],  # Add text for values
    textposition='auto'  # Automatically position the text
))

# Adding 'Deaths' data
fig.add_trace(go.Bar(
    x=top_provinces['Province_State'], 
    y=top_provinces['Deaths'], 
    name='Deaths', 
    marker_color='green',
    hoverinfo='x+y',  # Show Province_State and Deaths values on hover
    text=top_provinces['Deaths'],  # Add text for values
    textposition='auto'  # Automatically position the text
))

# Updating layout
fig.update_layout(
    title='Top 10 Provinces/States by Deaths: Confirmed Cases vs Deaths',
    xaxis=dict(title='Province/State'),
    yaxis=dict(title='Count'),
    barmode='group',  # Group the bars side by side
    template='plotly_white',
    hovermode='x'  # Enable hover mode on x-axis
)

# Show the figure
fig.show()












In [ ]:
#CODES FOR MODEL DEVELOPMENT

# Target variable selection
confirmed_cases = dfx['Confirmed']
#Deaths = dfx['Deaths']
# Time frame selection
train = confirmed_cases[:-60]  # Training data (all but last 30 observations)
test = confirmed_cases[-60:]   # Test data (last 30 observations)

# Verify datetime index
print(train.index)  # Should show datetime values
print(test.index)   # Should show datetime values

# SARIMA Model fitting
model = SARIMAX(train,
                order=(3, 1, 3),
                seasonal_order=(2, 0, 2, 7),
                enforce_stationarity=False,
                enforce_invertibility=False)

# Fit the model
model_fit = model.fit()

# Forecasting
forecast_steps = 60
forecast = model_fit.forecast(steps=forecast_steps)

# Generate correct forecast dates
forecast_dates = pd.date_range(start=test.index[-1], periods=forecast_steps + 1, freq='D')[1:]

# Plotting the results
plt.figure(figsize=(12, 6))
plt.plot(train.index, train, label='Training Data')  # Training data
plt.plot(test.index, test, label='Actual Data (Test)', color='green', linestyle='--')  # Test data
plt.plot(forecast_dates, forecast, label='Forecast', color='red')  # Forecasted data
plt.title('COVID-19 Confirmed Cases: Historical vs Forecasted')
plt.xlabel('Date', fontsize=12)
plt.ylabel('Number of Cases (millions)', fontsize=12)
plt.legend()
plt.show()



                  #MODEL DEVELOPMENT


# Target variable selection
total_death = dfx['Deaths']

# Time frame selection
train = total_death[:-60]  # Training data (all but last 30 observations)
test = total_death[-60:]   # Test data (last 30 observations)

# Verify datetime index
print(train.index)  # Should show datetime values
print(test.index)   # Should show datetime values

# SARIMA Model fitting
model = SARIMAX(train,
                order=(3, 1, 3),
                seasonal_order=(2, 0, 2, 7),
                enforce_stationarity=False,
                enforce_invertibility=False)

# Fit the model
model_fit = model.fit()

# Forecasting
forecast_steps = 60
forecast = model_fit.forecast(steps=forecast_steps)

# Generate correct forecast dates
forecast_dates = pd.date_range(start=test.index[-1], periods=forecast_steps + 1, freq='D')[1:]

# Plotting the results: Historical data vs forecasted data
plt.figure(figsize=(12, 6))
plt.plot(train.index, train, label='Training Data', color='blue')  # Training data
plt.plot(test.index, test, label='Actual Data (Test)', color='orange', linestyle='--')  # Test data
plt.plot(forecast_dates, forecast, label='Forecast', color='pink')  # Forecasted data
plt.title('COVID-19 Total Deaths: Historical vs Forecasted')  # Set the plot title
plt.xlabel('Date', fontsize=12)  # Label for x-axis
plt.ylabel('Number of Deaths', fontsize=12)  # Label for y-axis
plt.legend()
plt.grid()
plt.show()



#MODEL DEV FOR TOTAL RECOVERY

# Target variable selection
total_recovery = dfx['Recovered']

# Time frame selection
train =total_recovery[:-60]
test = total_recovery[-60:]

# Verify datetime index
print(train.index)  # Should show datetime values
print(test.index)   # Should show datetime values

# SARIMA Model fitting
model = SARIMAX(train,
                order=(3, 1, 3),
                seasonal_order=(2, 0, 2, 7),
                enforce_stationarity=False,
                enforce_invertibility=False)

# Fit the model
model_fit = model.fit()

# Forecasting
forecast_steps = 60
forecast = model_fit.forecast(steps=forecast_steps)

# Generate correct forecast dates
forecast_dates = pd.date_range(start=test.index[-1], periods=forecast_steps + 1, freq='D')[1:]

# Plotting the results: Historical data vs forecasted data
plt.figure(figsize=(12, 6))
plt.plot(train.index, train, label='Training Data', color='blue')  # Training data
plt.plot(test.index, test, label='Actual Data (Test)', color='orange', linestyle='--')  # Test data
plt.plot(forecast_dates, forecast, label='Forecast', color='pink')  # Forecasted data
plt.title('COVID-19 Total Recovery: Historical vs Forecasted')  # Set the plot title
plt.xlabel('Date', fontsize=12)  # Label for x-axis
plt.ylabel('Recovery', fontsize=12)  # Label for y-axis
plt.legend()
plt.grid()
plt.show()

                  









In [ ]:
#CODES FOR MODEL EVALUATION
# RMSE (Root Mean Squared Error) FOR MODEL EVALUATION
rmse = np.sqrt(mean_squared_error(test, forecast))
print(f'RMSE: {rmse}')

# MAE (Mean Absolute Error)
mae = mean_absolute_error(test, forecast)
print(f'MAE: {mae}')

# Plot residuals to check randomness
residuals = model_fit.resid
plt.figure(figsize=(12, 6))
plt.plot(residuals)
plt.title('Residuals')
plt.show()



# RMSE (Root Mean Squared Error)
rmse = np.sqrt(mean_squared_error(test, forecast))
print(f'RMSE: {rmse:.2f}')

# MAE (Mean Absolute Error)
mae = mean_absolute_error(test, forecast)
print(f'MAE: {mae:.2f}')

# R² Score
r2 = r2_score(test, forecast)
print(f'R² Score: {r2:.2f}')
# Mean Absolute Percentage Error (MAPE)
mape = np.mean(np.abs((test - forecast) / test)) * 100
print(f'MAPE: {mape:.2f}%')

# Normalized RMSE (NRMSE)
nrmse = rmse / test.mean()
print(f'Normalized RMSE: {nrmse:.2%}')
# Residuals
residuals = test - forecast
plt.figure(figsize=(12, 6))
plt.plot(residuals, label='Test Residuals', color='purple')
plt.axhline(0, color='red', linestyle='--', linewidth=1)
plt.title('Residuals (Test Set)')
plt.legend()
plt.show()
